<a href="https://colab.research.google.com/github/Mirnada/Covid_19/blob/master/Covid_19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análise dos dados do Covid_19


## Montando o Drive

In [10]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Importando bibliotecas

In [2]:
import pandas as pd

df = pd.read_csv('/content/drive/My Drive/Datasets/dados-curso-completo.csv')

## Dataset original - Estatísticas

### Informações gerias do dataset

In [ ]:
df.head(10)

,data,usuario,apelido,texto,retweet,seguidores,idioma,lugar,pais,sigla,latitude,longitude,hashtags
0,2020-03-23 18:51:14,Gaboardi,fraslee,@BolsonaroSP Cade o fundão deputado??? Congres...,0,138,pt,NaN,NaN,NaN,NaN,NaN,[]
1,2020-03-18 03:18:14,Seu Zé da Farmácia,seuzedafarmacia,"Em ação preventiva, o Governo do Estado suspen...",0,56,pt,NaN,NaN,NaN,NaN,NaN,[]
2,2020-03-29 12:09:27,Marília Oliveira 🇧🇷,Marioliveira_gb,Incrível como os defensores do fim do distanci...,0,5,pt,NaN,NaN,NaN,NaN,NaN,[]
3,2020-04-01 21:11:07,🚩,santosodeto,Fumaça de cigarro pode propagar coronavírus? h...,0,1885,pt,NaN,NaN,NaN,NaN,NaN,[]
4,2020-03-19 17:49:40,Nova Impressão,novaimpressao_,🚨Todos juntos contra COVID-19💪\n\nSolicite seu...,0,124,pt,Brasília,Brazil,BR,-47.8778,-15.77691,[]
5,2020-05-08 11:21:36,Marcel Silvano,marcelsilvano,"Importantes avanços! A pesquisa brasileira, a ...",4,2769,pt,Macaé,Brazil,BR,NaN,NaN,[]
6,2020-03-10 21:27:40,Teixeira Correia,LidadorNoticias,(Última hora) Volta ao Alentejo: Anulada por c...,0,39,pt,NaN,NaN,NaN,NaN,NaN,[]
7,2020-06-04 13:22:21,Thiago Moraes,ThiagoMoraesPB,João Pessoa tem queda no isolamento social por...,1,3578,pt,NaN,NaN,NaN,NaN,NaN,[]
8,2020-05-07 21:32:12,PT Brasil,ptbrasil,A #tvPT de hoje vai falar sobre a pandemia na ...,10,942696,pt,NaN,NaN,NaN,NaN,NaN,['tvPT']
9,2020-03-11 20:27:49,Niquito ;),NLazaroto,O Coronavirús causou uma pandemia e eu ainda n...,0,215,pt,NaN,NaN,NaN,NaN,NaN,[]


In [ ]:
# tamanho do dataset
# mais de 1,5 milhão de instancias e 13 atributos
df.shape

(1658825, 13)

In [ ]:
# quantidade de valores nulos
df.isnull().sum()

data                0
usuario            57
apelido             0
texto               0
retweet             0
seguidores          0
idioma              0
lugar         1584220
pais          1584225
sigla         1584230
latitude      1650105
longitude     1650105
hashtags            0
dtype: int64

### Descrição das colunas do Dataset

In [ ]:
# nomes das colunas
df.columns

Index(['data', 'usuario', 'apelido', 'texto', 'retweet', 'seguidores',
       'idioma', 'lugar', 'pais', 'sigla', 'latitude', 'longitude',
       'hashtags'],
      dtype='object')

* data: data do tweet do usuário
* usuario: nome do usuario
* apelido: apelido do usuário
* texto: texto publicado pelo usuário
* retweet: quantidade de vezes que o tweet foi replicado
* seguidores: quantidade de seguidores do usuário
* idioma: idioma do usuário
* lugar: lugar do usuário na publicação
* pais: país do usuário na publicação
* sigla: sigla do Estado do usuário
* latitude: posição geogŕafica do usuário 
* longitude: posição geográfica do usuário

In [ ]:
# verificando os tipos de dados do Dataset
df.dtypes

data           object
usuario        object
apelido        object
texto          object
retweet         int64
seguidores      int64
idioma         object
lugar          object
pais           object
sigla          object
latitude      float64
longitude     float64
hashtags       object
dtype: object

### Analisando os dados numericos

In [ ]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
retweet,1658825.0,197.853299,6337.641384,0.000000,0.000000,0.000000,0.000000,1.139763e+06
seguidores,1658825.0,32572.055193,368029.985029,0.000000,101.000000,380.000000,1470.000000,8.424125e+07
latitude,8720.0,-40.237583,20.808494,-123.120400,-47.864334,-43.955615,-38.453198,1.747893e+02
longitude,8720.0,-11.117100,19.169105,-37.818769,-23.305300,-19.624400,-7.120700,5.825778e+01


Ao gerar alguns dados estatísticos, podemos notas:

* a variável *retweet* tem como valor máximo 1139763, isso que dizer que uma publicação específica foi replicada mais de 1 milhão de vezes, assim como a variável *seguidores* nos mostra que um determinado usuário possui aproximadamente 8,5 milhões de seguidores 

* as colunas *retweet* e *seguidores* possui uma grande discrepância em sua distribuição de frequencia. Em *retweet*, todos os valores estão acima do 3º quartil, indicando que 75% dos valores são iguais a zero.


### Analisando os dados categóricos

In [ ]:
df['sigla'].value_counts()

Brazil           67247
Portugal          2675
United States     1068
Italy              484
Spain              379
                 ...  
Botswana             1
French Guiana        1
Macau                1
Chad                 1
Armenia              1
Name: pais, Length: 121, dtype: int64

Aproximadamente 90% dos dados são do Brasil

*Obs: apenas os dados que contem a sigla do país

In [ ]:
(df['sigla'].value_counts()['BR']/df['sigla'].value_counts().sum())*100

90.14947382532341

## Dataset DuvidasDB

In [13]:
duvidasDB = df[df['texto'].str.contains('?', regex=False)].copy()
duvidasDB.head(10)

,data,usuario,apelido,texto,retweet,seguidores,idioma,lugar,pais,sigla,latitude,longitude,hashtags
0,2020-03-23 18:51:14,Gaboardi,fraslee,@BolsonaroSP Cade o fundão deputado??? Congres...,0,138,pt,NaN,NaN,NaN,NaN,NaN,[]
3,2020-04-01 21:11:07,🚩,santosodeto,Fumaça de cigarro pode propagar coronavírus? h...,0,1885,pt,NaN,NaN,NaN,NaN,NaN,[]
38,2020-06-25 11:09:19,Andreia Peraro,AndreiaPeraro,@ipratesMD @dr_pampers Será que corremos o ris...,0,86,pt,NaN,NaN,NaN,NaN,NaN,[]
45,2020-04-06 15:44:36,Diorgenes Pandini,diorpandini,Prioridade ?!\n\n #COVID19 #floripa #isolament...,0,219,pt,NaN,NaN,NaN,NaN,NaN,"['COVID19', 'floripa', 'isolamento']"
57,2020-03-30 00:43:36,Valnei do Val,vdoval,"Que tal uma notícia boa, indo na contramão do ...",0,201,pt,NaN,NaN,NaN,NaN,NaN,[]
73,2020-04-07 23:19:44,Mauricio Ligabo,LigaboMauricio,@JoseMedeirosMT Seria isso algum tipo aleatóri...,0,23,pt,NaN,NaN,NaN,NaN,NaN,[]
77,2020-04-13 19:16:51,Gus,arrombagus,"""Nova York é o novo epicentro da pandemia de c...",3,253,pt,Santos,Brazil,BR,NaN,NaN,[]
116,2020-05-15 18:15:33,Silveacyr Maria Coelho Santos,silveacyr,@mitags Será por que o Bolsonaro não fez propa...,0,67,pt,NaN,NaN,NaN,NaN,NaN,[]
118,2020-04-08 22:04:51,lulleitao,lulleitao,@JeanSCorrea7 Mas ele não deu a entender que o...,0,2315,pt,NaN,NaN,NaN,NaN,NaN,[]
119,2020-04-02 13:04:55,Luizcarlosdospatetas@hotmail.com,Luizcarlosdosp5,#perguntacorona \nQuero saber se a covid tem c...,0,0,pt,NaN,NaN,NaN,NaN,NaN,['perguntacorona']


### Informações gerais do novo dataset

In [14]:
duvidasDB.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 172501 entries, 0 to 1658821
Data columns (total 13 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   data        172501 non-null  object 
 1   usuario     172495 non-null  object 
 2   apelido     172501 non-null  object 
 3   texto       172501 non-null  object 
 4   retweet     172501 non-null  int64  
 5   seguidores  172501 non-null  int64  
 6   idioma      172501 non-null  object 
 7   lugar       7829 non-null    object 
 8   pais        7829 non-null    object 
 9   sigla       7829 non-null    object 
 10  latitude    594 non-null     float64
 11  longitude   594 non-null     float64
 12  hashtags    172501 non-null  object 
dtypes: float64(2), int64(2), object(9)
memory usage: 18.4+ MB


### Informações dos temas discutidos

* Doença
* Medicamentos
* Tratamentos
* Pessoas

In [35]:
# lista de palavras chaves para cada título

doenca = ['codiv','zica','chikungunya','dengue', 'covid-19', 'covid19','Covid-19','CoronaVirus','coronavírus','Covid','Covid_19','GRIPESINHA','gripesinha','SARS','epidemia']
medicamento = ['cloroquina','hidroxicloroquina','Vermictina','novalgina']
organização = ['OMS','STF','PT','PF','Globo','CNN','GLOBO','CNNBrasil','Ministério','Banco','ONUBrasil','ONU','Anvisa']
pessoa = ['jair', 'bolsonaro', 'jair bolsonaro', 'presidente','BOZO','bozo','bosonaro','Mandetta','Trump','Lula','Doria','Haddad','mandetta','GilmarMendes','Queiroz']

In [36]:
doencaDB = duvidasDB[duvidasDB['texto'].str.contains(('|').join(doenca))].copy()
doencaDB.reset_index(drop=True, inplace=True)
a = doencaDB['texto']
a

0        Fumaça de cigarro pode propagar coronavírus? h...
1        @JoseMedeirosMT Seria isso algum tipo aleatóri...
2        E essa chuvinha boa para pegar um resfriado e ...
3        Você iria na festa de aniversário de @jairbols...
4        Hotel desaba na China com 70 pessoas em quaren...
                               ...                        
29388    Come posso esservi inutile?\n#primavera2020 #c...
29389    Animais domésticos podem pegar ou transmitir o...
29390    @0n3p1ece @alexmono Oh jumentinha, 1918 é isso...
29391    @BiologiaBraba @Tati_Cordeiro Só vontade ou vo...
29392    @OsmarTerra Sr. Osmar Terra, gostaria de ouvir...
Name: texto, Length: 29393, dtype: object

In [37]:
medicaDB = duvidasDB[duvidasDB['texto'].str.contains(('|').join(medicamento))].copy()
medicaDB.reset_index(drop=True, inplace=True)
b = medicaDB['texto']
b

0        @bbcbrasil @cavmota A pergunta que não quer ca...
1        @gilmarmendes Ué ?! Mas a cloroquina não recom...
2        Será que a Renata vai tomar cloroquina William...
3        @jdoriajr O que vc tem contra a cloroquina? At...
4        @BgwElectric @Leitadas_L0en @TeichNelson Márci...
                               ...                        
16881    Mais essa tal de cloroquina não é aquela que m...
16882    @jairbolsonaro Você fez? Vai assumir que está ...
16883    QUAIS os países estão usando cloroquina para t...
16884    @saviscos @dinhomidias @helderbarbalho Nossa q...
16885    @OsmarTerra Sr. Osmar Terra, gostaria de ouvir...
Name: texto, Length: 16886, dtype: object

In [38]:
pessoaDB = duvidasDB[duvidasDB['texto'].str.contains(('|').join(pessoa))].copy()
pessoaDB.reset_index(drop=True, inplace=True)
e = pessoaDB['texto']
e

0        Você iria na festa de aniversário de @jairbols...
1        É sério que o presidente mandou os governadore...
2        @jairbolsonaro \nComo vc demite uma pessoa que...
3        @jairbolsonaro @jairbolsonaro, você não conseg...
4        @Inter_desde1988 @alex3555 @Modinhaseculo21 @j...
                               ...                        
11997    @jairbolsonaro Se tocaram que descumprir o Jur...
11998    @jairbolsonaro Você fez? Vai assumir que está ...
11999    Ansioso pelas palavras do presidente referente...
12000    Mundo no meio de uma pandemia, e o excelentíss...
12001    @lhmandetta o exame do Jair Bolsonaro é positi...
Name: texto, Length: 12002, dtype: object

In [39]:
orgDB = duvidasDB[duvidasDB['texto'].str.contains(('|').join(organização))].copy()
orgDB.reset_index(drop=True, inplace=True)
d = orgDB['texto']
d

0       @gilmarmendes Ué ?! Mas a cloroquina não recom...
1       Será que a Renata vai tomar cloroquina William...
2       @amorimvivian_ GIIIIII ??? TÁ SE CUIDANDO DIRE...
3       @CNNBrasil Só a Cloroquina já derruba os dois ...
4       @Estadao @lobaoeletrico OMS diz que ciência pr...
                              ...                        
6291    @radiotupi NÃO seria culpa da GLOBO?! OU SERIA...
6292    #STF @jairbolsonaro PQP o STF falando em solta...
6293    Urgente: OMS declara pandemia mundial do coron...
6294    O tratamento com a cloroquina está sendo utili...
6295    #GrandeDebateCNN Tiago Anastácio, falou q falt...
Name: texto, Length: 6296, dtype: object

### Analisando o texto

In [5]:
textos = df['texto']
print (textos)

0          @BolsonaroSP Cade o fundão deputado??? Congres...
1          Em ação preventiva, o Governo do Estado suspen...
2          Incrível como os defensores do fim do distanci...
3          Fumaça de cigarro pode propagar coronavírus? h...
4          🚨Todos juntos contra COVID-19💪\n\nSolicite seu...
                                 ...                        
1658820    oms: tamo tendo pandemia evitar\n conglomerado...
1658821    @OsmarTerra Sr. Osmar Terra, gostaria de ouvir...
1658822    CIENTISTA DE HARVARD PREVÊ QUE ATÉ 70% DO MUND...
1658823    #BolsonaroTemRazaoSim porque, quando a prefeit...
1658824    Em meio a uma pandemia, estamos na luta também...
Name: texto, Length: 1658825, dtype: object


###Quebrando o texto em palavras

In [6]:
texto_quebra = textos.str.split()
print (texto_quebra)

0          [@BolsonaroSP, Cade, o, fundão, deputado???, C...
1          [Em, ação, preventiva,, o, Governo, do, Estado...
2          [Incrível, como, os, defensores, do, fim, do, ...
3          [Fumaça, de, cigarro, pode, propagar, coronaví...
4          [🚨Todos, juntos, contra, COVID-19💪, Solicite, ...
                                 ...                        
1658820    [oms:, tamo, tendo, pandemia, evitar, conglome...
1658821    [@OsmarTerra, Sr., Osmar, Terra,, gostaria, de...
1658822    [CIENTISTA, DE, HARVARD, PREVÊ, QUE, ATÉ, 70%,...
1658823    [#BolsonaroTemRazaoSim, porque,, quando, a, pr...
1658824    [Em, meio, a, uma, pandemia,, estamos, na, lut...
Name: texto, Length: 1658825, dtype: object


###Analisando o primeiro indice 

In [ ]:
print (texto_quebra[0])

['@BolsonaroSP', 'Cade', 'o', 'fundão', 'deputado???', 'Congresso', 'está', 'sentado', 'em', '3BI', 'olhando', 'a', 'pandemia', 'da', 'janela.']


Contando a quantidade de palavras do primeiro indice

In [ ]:
len(texto_quebra[0])

15

In [ ]:
data = df.texto
print (data[0])
inicio = data[0].find('?')
print (inicio)
#fim = data[0].find('', inicio)
#print (fim)
#host = data[inicio +1 : fim]
#print (host)

@BolsonaroSP Cade o fundão deputado??? Congresso está sentado em 3BI olhando a pandemia da janela.
35


###Percorrendo os elementos da Lista 

In [ ]:
palavras = df.texto
i = 0
while i < len(palavras):
  print (palavras[i])
  i = i + 1


###Usando a função enumerate para visualizar o conjunto de palavras

In [ ]:
for cont in enumerate(texto_quebra): 
  print (cont)